In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm
from IPython.display import display, Markdown

In [2]:
%matplotlib inline
mpl.rc('font', size=14)
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['text.usetex'] = True 
mpl.rcParams['axes.grid'] = True

In [3]:
np.random.seed(42)

In [4]:
def generate(sigma_gt, sigma_Lt, sigma_thetat, N, true_g=9.80665, true_L=1., true_theta_0=10):
    gs = np.random.normal(true_g, sigma_gt, N)
    Ls = np.random.normal(true_L, sigma_Lt, N)
    thetas = np.deg2rad(np.random.uniform(true_theta_0 - sigma_thetat, 
                                          true_theta_0 + sigma_thetat, N))
    return gs, Ls, thetas

def period_int(g, L, theta_0, sigma_n, N=int(3 * 1e7)):
    thetas = np.linspace(0, theta_0, N)[:-1]
    integrand = 1 / np.sqrt(np.cos(thetas) - np.cos(theta_0))
    integral = np.trapz(integrand, thetas)
    per = 2 * np.sqrt(2) * np.sqrt(L / g) * integral
    sys_unc = np.random.uniform(-sigma_n, sigma_n)
    return per + sys_unc

def find_g(ts, Ls):
    return 4 * np.pi**2 * Ls / ts**2

def dg_dt(ts, Ls):
    return - (8 * np.pi**2 * Ls) / ts**3

def dg_dL(ts):
    return (4 * np.pi**2) / ts**2

def find_g_err(ts, Ls, sigma_ti, sigma_Li):
    t_unc = dg_dt(ts, Ls)**2 * sigma_ti**2
    L_unc = dg_dL(ts)**2 * sigma_Li**2
    return np.sqrt(t_unc + L_unc)

In [16]:
def weighted_mean(vals, uncs):
    weights = 1 / uncs**2
    mean = np.sum(weights * vals) / np.sum(weights)
    unc = 1 / np.sqrt(np.sum(weights))
    return mean, unc

def display_vals(params, g_mean, g_unc):
    display(Markdown(f'<h3>${[param for param in params]}$<h3>'))
    display(Markdown(f'<h3>$g = {g_mean:0.5f} \pm {g_unc:0.5f}$<h3>'))

In [22]:
def run_sim(params, N=50, disp=True):
    """
    Calculates the mean measurement and uncertainty on g, given a set of uncertainties.
    
    Parameters
    ----------
        params : list
            params[0], the uncertainty on the ``true'' value of g
            params[1],          "         "          "          L
            params[2],          "         "          "          theta
            params[3],          "         "   measured period
            params[4], the precision of the stopwatch/timer
            params[5],          "         " ruler
    
    """
    sigma_gt, sigma_Lt, sigma_thetat, sigma_n, sigma_ti, sigma_Li = params
    gs, Ls, thetas = generate(sigma_gt, sigma_Lt, sigma_thetat, N)
    ts = np.zeros(N)
    for i, (g, L, theta) in enumerate(tqdm(zip(gs, Ls, thetas), total=N)):
        ts[i] = period_int(g, L, theta, sigma_n)
    g_i = find_g(ts, Ls)
    g_i_unc = find_g_err(ts, Ls, sigma_ti, sigma_Li)
    g_mean, g_unc = weighted_mean(g_i, g_i_unc)
    if disp:
        display_vals(params, g_mean, g_unc)
#     return g_mean, g_unc

In [23]:
vals = [
    [0.000001, 1e-6, 0.5, 0.15, 0.01, 0.001], 
    [0., 1e-6, 0.5, 0.15, 0.01, 0.001],
    [0.000001, 0., 0.5, 0.15, 0.01, 0.001],
    [0.000001, 1e-6, 0., 0.15, 0.01, 0.001],
    [0.000001, 1e-6, 0.5, 0., 0.01, 0.001],
    [0.000001, 1e-6, 0.5, 0.15, 0., 0.001],
    [0.000001, 1e-6, 0.5, 0.15, 0.01, 0.],
    [0., 0., 0., 0.15, 0.01, 0.001],
    [0., 0., 0., 0.01, 0.01, 0.0001],
]

In [24]:
for val in vals:
    print(u'\u2500' * 60)
    run_sim(val)

────────────────────────────────────────────────────────────


  0%|          | 0/50 [00:00<?, ?it/s]

<h3>$[1e-06, 1e-06, 0.5, 0.15, 0.01, 0.001]$<h3>

<h3>$g = 9.57688 \pm 0.01356$<h3>

────────────────────────────────────────────────────────────


...

<h3>$g = 9.81584 \pm 0.01405$<h3>

────────────────────────────────────────────────────────────


  0%|          | 0/50 [00:00<?, ?it/s]

<h3>$[0.0, 0.0, 0.0, 0.01, 0.01, 0.0001]$<h3>

<h3>$g = 9.75694 \pm 0.01372$<h3>